In [47]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import time

In [24]:
def clean_label(label):
    return label.lstrip(',').rstrip(',').replace(',,', ',')
INPUT_FOLDER = 'processed_data/'
MAIN_FOLDER = "Data/"
labels_file = 'labels.txt'
headers_metadf = ['trajectory_id', 'start_time', 'end_time', 'v_ave', 'v_med', 'v_max', 'v_std', 'a_ave', 'a_med', 'a_max', 'a_std', 'labels']

list_df_metadata = []

def list_of_subfolders_with_label(path):
    folder_list = []
    directories = os.listdir(path)
    for subfolder in directories:
        number = subfolder
        subfolder = path + subfolder + '/'
        if(labels_file in os.listdir(subfolder)):
            folder_list.append(number)
    return folder_list

folder_list = list_of_subfolders_with_label(MAIN_FOLDER)
folder_list

['010',
 '020',
 '021',
 '052',
 '053',
 '056',
 '058',
 '059',
 '060',
 '062',
 '064',
 '065',
 '067',
 '069',
 '073',
 '075',
 '076',
 '078',
 '080',
 '081',
 '082',
 '084',
 '085',
 '086',
 '087',
 '088',
 '089',
 '092',
 '096',
 '097',
 '098',
 '100',
 '101',
 '102',
 '104',
 '105',
 '106',
 '107',
 '108',
 '110',
 '111',
 '112',
 '068',
 '091',
 '114',
 '115',
 '116',
 '117',
 '118',
 '124',
 '125',
 '126',
 '128',
 '129',
 '136',
 '138',
 '139',
 '141',
 '144',
 '147',
 '153',
 '154',
 '161',
 '163',
 '167',
 '170',
 '174',
 '175',
 '179']

In [33]:
for file in folder_list:
    if int(file) == 161:
        filename = file + "_2.csv"
        df_metadata = pd.read_csv(filename, index_col=0)
        list_df_metadata.append(df_metadata)
df_metadata = pd.concat(list_df_metadata)
df_metadata.head()

,lat,long,altitude,datetime,timedelta,distance,velocity,acceleration,trajectory_id,subfolder,labels,v_ave,v_med,v_max,a_ave,a_med,a_max
0,39.974300,116.328117,695.538058,2007-04-17 13:01:53,0 days 00:00:10.000000000,14.710840,1.471084,0.036022,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
1,39.974333,116.327950,616.797900,2007-04-17 13:02:03,0 days 00:00:12.000000000,21.975597,1.831300,-0.141486,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
2,39.974417,116.328183,702.099738,2007-04-17 13:02:15,0 days 00:02:34.000000000,20.554457,0.133470,0.073932,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
3,39.974233,116.328150,606.955381,2007-04-17 13:04:49,0 days 00:00:09.000000000,103.670968,11.518996,-0.841308,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
4,39.973300,116.328183,606.955381,2007-04-17 13:04:58,0 days 00:20:07.000000000,4764.305281,3.947229,-0.001730,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932


In [35]:
df_labeled = df_metadata.dropna(subset=['v_ave','v_med','v_max',  'a_ave', 'a_med', 'a_max',  'labels'])

In [36]:
df_labeled.loc[:,'labels'] = df_labeled['labels'].apply(lambda x: clean_label(x))

/home/yogesh/miniconda3/envs/slash/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [38]:
df_labeled.head()

,lat,long,altitude,datetime,timedelta,distance,velocity,acceleration,trajectory_id,subfolder,labels,v_ave,v_med,v_max,a_ave,a_med,a_max
0,39.974300,116.328117,695.538058,2007-04-17 13:01:53,0 days 00:00:10.000000000,14.710840,1.471084,0.036022,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
1,39.974333,116.327950,616.797900,2007-04-17 13:02:03,0 days 00:00:12.000000000,21.975597,1.831300,-0.141486,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
2,39.974417,116.328183,702.099738,2007-04-17 13:02:15,0 days 00:02:34.000000000,20.554457,0.133470,0.073932,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
3,39.974233,116.328150,606.955381,2007-04-17 13:04:49,0 days 00:00:09.000000000,103.670968,11.518996,-0.841308,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932
4,39.973300,116.328183,606.955381,2007-04-17 13:04:58,0 days 00:20:07.000000000,4764.305281,3.947229,-0.001730,20070417130153,161,walk,1.775769,1.424775,11.518996,-0.028917,-0.003264,0.073932


In [42]:
mask = np.random.rand(len(df_labeled)) < 0.7
df_train = df_labeled[mask]
df_test = df_labeled[~mask]

print(len(df_train), len(df_test))

947 373


In [43]:
X_colnames = ['v_ave','v_med','v_max', 'a_ave', 'a_med', 'a_max']
Y_colnames = ['labels']

X_train = df_train[X_colnames].values
Y_train = np.ravel(df_train[Y_colnames].values)
X_test = df_test[X_colnames].values
Y_test = np.ravel(df_test[Y_colnames].values)

## Machine Learning Model

In [44]:
rf = RandomForestClassifier(n_estimators = 18)
svm = SVC()

In [45]:
# Random forest parameter tuning search
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [52]:
#Random Forest
t_start = time.process_time()
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, Y_train)
t_end = time.process_time()
t_diff = t_end - t_start
print(t_diff)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.4min finished


1.9874110079999996


In [67]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    print("Prediction", predictions, test_labels)
#     errors = abs(predictions - test_labels)
    count = 0
    errors = 0
    for i in predictions:
        if i != test_labels[count]:
            errors = errors +1
        count = count +1
    mape = 100 * np.mean(errors / len(test_labels))
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} '.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [70]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)

Prediction ['walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'bus' 'bus' 'bus' 'bus' 'bus'
 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus'
 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'bike' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'taxi' 'bus' 'bus' 'bus' 'bus' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'bus' 'bus' 'bus' 'bus' 'bus'
 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'bus' 'b